In [2]:
from __future__ import print_function
import os
import numpy as np
from numpy import newaxis
from sklearn import preprocessing
np.random.seed(1337)

import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Dropout
from keras.models import Model,Sequential
import sys

import pandas as pd
import random

Using TensorFlow backend.


# Reading in data

In [3]:
print("Data read started...")
data = pd.read_csv("result_nback_tetris.csv")
data = data.as_matrix()
print ("Data read finished.")

print(data.shape)

Data read started...
Data read finished.
(1125, 11)


# Eliminate EEG data

In [4]:
for i in range (1,5):
    data = np.delete(data, 1, 1) 

data.shape

(1125, 7)

In [5]:
data[1]

array([3, 1318300.0, 1839.9, 673.37, 4.746279069767441, 371.67, 522.54], dtype=object)

# Z normalize

In [6]:
for i in range (1,7):
    data[:, i] = preprocessing.scale(data[:, i])

/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConve

In [7]:
x_data = data[:, 1:]
y_data = data[:, 0]

# One-hot encoding

In [8]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y_one_hot = ohe.fit_transform(y_data.reshape(-1,1)).toarray()
print(y_one_hot)

[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 ..., 
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]


# Shuffle data

In [9]:
indices = np.arange(x_data.shape[0])
np.random.shuffle(indices)

x_data = x_data[indices]
y_one_hot = y_one_hot[indices]

# Divide into train, validation and test data

In [10]:
len_data = len(x_data)

nb_test = int(len_data*0.15)
nb_validation = int(len_data*0.15)
nb_train = int(len_data*0.7)

end_valid = nb_train+nb_validation

x_train = x_data[0:nb_train]
y_train = y_one_hot[0:nb_train]

x_valid = x_data[nb_train:end_valid]
y_valid = y_one_hot[nb_train:end_valid]

x_test = x_data[end_valid:]
y_test = y_one_hot[end_valid:]

In [11]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(787, 6) (787, 3) (168, 6) (168, 3) (170, 6) (170, 3)


# Build the net

In [12]:
model = Sequential()

model.add(Dense(200, input_shape=(6,)))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(3, activation='softmax'))

In [13]:
batch_size = 50
epochs = 150
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='auto')

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(x_train, y_train, nb_epoch=epochs,batch_size=batch_size, 
          callbacks=[earlyStopping], shuffle=True, validation_data = (x_valid, y_valid))

Train on 787 samples, validate on 168 samples
Epoch 1/150
787/787 [==============================] - 0s - loss: 1.0371 - acc: 0.4562 - val_loss: 0.9754 - val_acc: 0.5357
Epoch 2/150
787/787 [==============================] - 0s - loss: 0.9718 - acc: 0.5451 - val_loss: 0.9082 - val_acc: 0.6250
Epoch 3/150
787/787 [==============================] - 0s - loss: 0.9550 - acc: 0.5604 - val_loss: 0.9013 - val_acc: 0.6131
Epoch 4/150
787/787 [==============================] - 0s - loss: 0.9218 - acc: 0.5693 - val_loss: 0.8192 - val_acc: 0.6012
Epoch 5/150
787/787 [==============================] - 0s - loss: 0.8654 - acc: 0.5832 - val_loss: 0.7863 - val_acc: 0.6429
Epoch 6/150
787/787 [==============================] - 0s - loss: 0.8468 - acc: 0.5883 - val_loss: 0.7370 - val_acc: 0.6369
Epoch 7/150
787/787 [==============================] - 0s - loss: 0.8317 - acc: 0.5896 - val_loss: 0.7107 - val_acc: 0.6845
Epoch 8/150
787/787 [==============================] - 0s - loss: 0.7893 - acc: 0.6264

In [14]:
model.evaluate(x_test, y_test, batch_size=batch_size)

150/170 [=========================>....] - ETA: 0s

[0.27207818276741924, 0.90588235855102539]

In [15]:
from sklearn.metrics import classification_report,confusion_matrix

y_pred = model.predict_classes(x_test)
print(y_pred)

p=model.predict_proba(x_test)

target_names = ['class 0(level1)', 'class 1(level2)', 'class 2(level3)']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

170/170 [==============================] - 0s     
[2 0 2 1 1 1 0 0 2 1 0 1 1 0 2 2 2 2 0 2 0 0 1 0 0 2 2 1 2 0 1 0 2 0 1 2 0
 1 0 1 2 1 2 2 0 1 2 0 0 2 2 1 1 2 0 2 1 1 2 0 0 0 2 2 2 1 0 1 0 2 0 2 0 1
 1 2 2 2 1 2 2 0 2 2 1 0 0 2 2 2 2 1 1 0 2 2 0 0 0 0 0 0 2 2 2 0 2 0 0 0 2
 0 0 1 1 2 2 1 2 2 0 2 2 1 2 0 2 2 1 1 0 0 0 2 2 2 2 1 2 1 1 1 0 0 1 1 2 0
 2 0 2 0 0 2 1 2 2 1 0 2 2 1 1 0 2 0 0 0 1 1]
 96/170 [===============>..............] - ETA: 0s                 precision    recall  f1-score   support

class 0(level1)       0.93      0.98      0.96        55
class 1(level2)       0.93      0.82      0.87        50
class 2(level3)       0.87      0.91      0.89        65

    avg / total       0.91      0.91      0.91       170

[[54  1  0]
 [ 0 41  9]
 [ 4  2 59]]
